In [2]:
import json
from typing import List
import requests

In [9]:
def apify_training_data(skills: List[str], suffix: str = ''):
    output_urls = []
    for s in skills:
        res = requests.post(
            'https://api.apify.com/v2/acts/G4B5HdLYCXDyiafhy/runs?token=BDR3StxrybGSpod9z5Ara2o83',
            json={
                'q': f'{s} {suffix}'
            }
        )

        kv_store = res.json()['data']['defaultKeyValueStoreId']
        print(kv_store)
        output_url = f"https://api.apify.com/v2/key-value-stores/{kv_store}/records/OUTPUT?disableRedirect=true"
        output_urls.append({'name': s, 'output_url': output_url})
    return output_urls

In [10]:
skills_terms = ['azure cognitive services', 'azure video indexer', 'azure text analytics', 'azure speech', 'azure computer vision']

In [13]:
data = [requests.get(o['output_url']).json() for o in output_urls]

In [14]:
with open('../data/processed/azure_bing_search_examples_cog_services.jsonl', 'w+') as f:
    for svc in data:
        svc_data = svc['webPages'] + svc['news']
        for svc_data in svc_data:
            if 'paragraphs' in svc_data:
                for p in svc_data['paragraphs']:
                    f.write(json.dumps({
                        'text': p,
                        'meta': {
                            'source_url': svc_data['url']
                        }
                    }) + '\n')